In [2]:
# Import library
import pandas as pd
import sqlite3

In [3]:
# Données du Guide de l'auto
df = pd.read_hdf('C:\\Users\\Charles_tour\\Documents\\GitHub\\car_sales_forecast\\Dataframe\\scrap_data.h5')
df.head(3)

,marque,modele,annee,spec,pdsf,moteur,type_carbu,type_vehicule,classement,eff_ville,eff_autoroute,eff_combi,eff_autonomie,eff_ges,masse,lien
0,volvo,xc70,2009,5prt-wgn-3.0-l-avec-toit-ouvrant,34350$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"7,9L/100km","6,3L/100km","7,2L/100km",638km,167g/km,1402kg (3091lb),https://www.guideautoweb.com/constructeurs/vol...
1,volvo,xc70,2009,5prt-wgn-3.0-l-avec-toit-ouvrant,37050$,"4L 1,5l",Super,Hatchback,Sous-compacte de luxe,"8,1L/100km","6,5L/100km","7,4L/100km",621km,172g/km,1415kg (3120lb),https://www.guideautoweb.com/constructeurs/vol...
2,volvo,xc70,2009,5prt-wgn-3.0-l-avec-toit-ouvrant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.guideautoweb.com/constructeurs/vol...


In [4]:
df_clean = df.copy()


# for index, row in df_clean.head().iterrows():
#     print(row)


df_clean = df_clean[['marque', 'modele', 'annee', 'eff_combi', 'eff_ges']] 
df_clean = df_clean.iloc[0:51]   # pour tester

# Refaire rouler sur cette ordi là
# Créer une nouvelle colonne qui distingue le type de drivetrain (ESS, VEB, HYB, VHB)
# Faire un groupby sur 'marque', 'modele', 'annee' pour l'efficacité et les émissions 

df_clean.head()

,marque,modele,annee,eff_combi,eff_ges
0,volvo,xc70,2009,"7,2L/100km",167g/km
1,volvo,xc70,2009,"7,4L/100km",172g/km
2,volvo,xc70,2009,NaN,NaN
3,volvo,xc70,2009,"7,4L/100km",172g/km
4,volvo,xc70,2009,"7,8L/100km",181g/km


In [5]:
# Nettoyer les données
df_clean['eff'] = df_clean['eff_combi'].str.replace(',', '.',regex=True).str.replace('L/100km', '',regex=True).str.replace('n.d.','0',regex=True).fillna(0).astype(float)
df_clean['ges'] = df_clean['eff_ges'].str.replace('g/km', '',regex=True).str.replace('n.d.','0',regex=True).fillna(0).astype(int)
df_clean = df_clean.drop(columns={'eff_combi', 'eff_ges'})

df_clean.head()

,marque,modele,annee,eff,ges
0,volvo,xc70,2009,7.2,167
1,volvo,xc70,2009,7.4,172
2,volvo,xc70,2009,0.0,0
3,volvo,xc70,2009,7.4,172
4,volvo,xc70,2009,7.8,181


In [9]:
# Pour tester
df_clean.loc[2, 'marque'] = 'Honda'
df_clean.loc[3, 'marque'] = 'Acura'

In [10]:
test.head()

,marque,modele,annee,eff,ges
0,volvo,xc70,2009,7.2,167
1,volvo,xc70,2009,7.4,172
2,Honda,xc70,2009,0.0,0
3,volvo,xc70,2009,7.4,172
4,volvo,xc70,2009,7.8,181


In [26]:
pd.set_option('display.max_columns', 1000)
for link in df.iloc[0:3,-1]:
    print(link)

https://www.guideautoweb.com/constructeurs/volvo/xc70/2009/specifications/5prt-wgn-3.0-l-avec-toit-ouvrant/
https://www.guideautoweb.com/constructeurs/volvo/xc70/2009/specifications/5prt-wgn-3.0-l-avec-toit-ouvrant/
https://www.guideautoweb.com/constructeurs/volvo/xc70/2009/specifications/5prt-wgn-3.0-l-avec-toit-ouvrant/


In [37]:
df_clean.columns.tolist()

['marque', 'modele', 'annee', 'eff', 'ges']

In [35]:
# Store dans une db
connection = sqlite3.connect('scrap_data.db')
cursor = connection.cursor()

df_clean.to_sql('vehicles', connection, if_exists='replace', index=False)

connection.commit()
connection.close()

In [36]:
# Accèder au db
connection = sqlite3.connect('scrap_data.db')
cursor = connection.cursor()

cursor.execute('SELECT * FROM vehicles')

# Columns name
column_names = [description[0] for description in cursor.description]
print("Column Names:", column_names)

# Values
rows = cursor.fetchall()
for row in rows:
    print(row)

connection.close()

Column Names: ['marque', 'modele', 'annee', 'eff', 'ges']
('volvo', 'xc70', '2009', 7.2, 167)
('volvo', 'xc70', '2009', 7.4, 172)
('volvo', 'xc70', '2009', 0.0, 0)
('volvo', 'xc70', '2009', 7.4, 172)
('volvo', 'xc70', '2009', 7.8, 181)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 12.4, 291)
('volvo', 'xc70', '2009', 12.4, 291)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 11.2, 263)
('volvo', 'xc70', '2009', 12.4, 291)
('volvo', 'xc70', '2009', 12.4, 291)
('volvo', 'xc70', '2009', 10.9, 254)
('volvo', 'xc70', '2009', 10.9, 254)
('volvo', 'xc70', '2009', 11.0, 258)
('volvo', 'xc70', '2009', 10.9, 254)
('volvo', 'xc70', '2009', 10.9, 254)
('volvo', 'xc70', '2009', 9.0, 210)
('volvo', 'xc70', '2009', 10.8, 252)
('volvo', 'xc70', '2009', 10.8, 252)
('volvo', 'xc70', '2009',